In [1]:
import pandas as pd
import numpy as np

import pickle
import json

In [2]:
train_df = pd.read_csv('./output/中間データ/test_preprocessed.csv')
train_df.head()

,datetime,y,week,soldout,name,kcal,remarks,payday,weather,precipitation,...,y_rolling_mean_14,y_rolling_max_14,y_rolling_min_14,day,kcal_missing_flag,curry,popular,unpopular,event_キャリアアップ支援セミナー,event_ママの会
0,2014-10-01,NaN,0.416215,1,メンチカツ,-0.079503,0,0.0,NaN,-0.313559,...,54.285714,115.0,29.0,-1.725325,0,0,1,0,False,False
1,2014-10-02,NaN,-0.871488,0,バーベキューチキン,-0.446438,0,0.0,NaN,-0.340302,...,NaN,NaN,NaN,-1.611442,0,0,0,0,False,False
2,2014-10-03,NaN,-0.843645,0,豚肉のマスタード焼き,-1.180308,0,0.0,NaN,-0.340302,...,NaN,NaN,NaN,-1.497560,0,0,0,0,False,False
3,2014-10-06,NaN,2.197171,1,麻婆春雨,-1.547242,0,0.0,NaN,1.023580,...,NaN,NaN,NaN,-1.155911,0,0,0,0,False,False
4,2014-10-07,NaN,0.088051,0,厚揚げ肉みそ炒め,0.654367,0,0.0,NaN,-0.340302,...,NaN,NaN,NaN,-1.042028,0,0,0,0,False,False


In [3]:
X_test = train_df.drop(columns=['y', 'name', 'datetime'])

In [5]:
with open('./output/モデル/lgb_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

new_y_pred = loaded_model.predict(X_test)

#### 予測結果の出力

In [6]:
result_df = pd.read_csv('./input/sample.csv', header=None)

In [8]:
# トレンド係数の読み込み
with open('./output/中間データ/trend_params.json', 'r') as f:
    trend_params = json.load(f)

a1, b1, c1 = trend_params['a'], trend_params['b'], trend_params['c']
trend_pred = a1 * np.exp(-b1 * X_test.index.values) + c1

y_pred = new_y_pred + trend_pred

In [9]:
result_df.iloc[:, 1] = y_pred

/var/folders/sb/t91v_nzj05sbsfnhcjb832xr0000gn/T/ipykernel_12524/3961981253.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[123.1078825  121.07795428 120.37117918 120.57235342 119.31615905
 118.48748328 116.89313936 119.30640689 115.97370638 115.63339465
 113.72431283 112.93662706 113.25912889 112.2583934  111.81421224
 110.02277263 109.95018468 109.56895234 109.02862292 107.93284521
 106.3731505  105.83151791 106.05375513 105.61241629 103.84114261
 106.24138645 103.33711413 102.72839999 102.06913828 100.5730099
  99.84400363 100.55703052  99.6669908   99.01694244  97.37441486
  99.56213271  97.24896935  96.63087398  94.97412309  94.40232917]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df.iloc[:, 1] = y_pred


In [10]:
result_df.to_csv('output/予測結果/予測結果.csv', index=False)